In [11]:
import cv2
import os
import base64
import requests
import pickle
import numpy as np

In [12]:
url = "http://localhost:8080/api/gethog"

# type
classB = ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce','Swift', 'Tata Safari', 'Toyota Innova']

def img2HOG(img):
    v, buffer = cv2.imencode(".jpg", img)
    img_str = base64.b64encode(buffer)
    data = "image data,"+str.split(str(img_str),"'")[1]
    response = requests.get(url, json={"img":data})
    
    return response.json()

In [13]:
# Read data function
def readData(path):
    response = []
    for sub in os.listdir(path):
        for fn in os.listdir(path + '/' + sub):
            img_file_name = path + '/' + sub + "/" + fn
            img = cv2.imread(img_file_name)
            res = img2HOG(img)
            hog = list(res["hog"])
            hog.append(classB.index(sub))
            response.append(hog)
    return response

In [9]:
# save data to pkl function
def savePkl(filename ,path):
    cars = readData(path)
        
    write_path = filename + ".pkl"
    pickle.dump(cars, open(write_path,"wb"))
    print("data preparation is done")

# load data to pkl function    
def loadPkl(filename):
    dataset = pickle.load(open(filename + '.pkl','rb'))
    return dataset

In [17]:
# เรียกใช้ function savePkl โดยส่ง path ของโฟลเดอร์ train ไป
train_dir = r'Cars Dataset\\train'
savePkl('train_cars',train_dir)

data preparation is done


In [18]:
# เรียกใช้ function savePkl โดยส่ง path ของโฟลเดอร์ test ไป
test_dir = r'Cars Dataset\test'
savePkl('test_cars',test_dir)

data preparation is done


In [19]:
# เรียกใช้ function loadPkl โดยส่งชื่อไฟล์ไป
dataset_train = loadPkl('train_cars')
print("Data train : ",len(dataset_train))
dataset_test = loadPkl('test_cars')
print("Data test : ",len(dataset_test))

Data train :  3352
Data test :  813


In [20]:
# แปลง list ข้อมูล train เป็น array
train_arr = np.array(dataset_train)
x_train = train_arr[:,0:-1]
y_train = train_arr[:,-1]

In [21]:
# แปลง list ข้อมูล test เป็น array
test_arr = np.array(dataset_test)
x_test = test_arr[:,0:-1]
y_test = test_arr[:,-1]

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [23]:
# เทรนข้อมูลด้วย Decision Tree
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [24]:
# ทดสอบประสิทธิภาพ
y_pred = clf.predict(x_test)
print("Accuracy:",accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix : \n",confusion_matrix(y_test,y_pred))

Accuracy: 47.23247232472325
Confusion Matrix : 
 [[ 73  13  12  32  22  21  26]
 [  8  38   3   0   3   4  11]
 [ 16   3  24   7   2   7  16]
 [ 26   2   8  15   5   9   9]
 [ 10   6   8   6  50   3  19]
 [ 13   3   2   9   7  66   6]
 [ 27   7   8  10  12   8 118]]


In [18]:
# บันทึก model file ด้วย pickle
path_model ='carmodel.pkl'
pickle.dump(clf, open(path_model,'wb'))